In [ ]:
import time, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import requests

burl = "https://www.volcanodiscovery.com/"
url = "https://www.volcanodiscovery.com/earthquakes/portugal/largest.html"

headers = {"User-Agent":"bingbot"}

response = requests.get(url, headers=headers)
stat = response.status_code
soup = BeautifulSoup(response.content, 'html.parser')

stat




In [ ]:
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source

In [ ]:
soup_s = BeautifulSoup(html)

In [ ]:
items = soup_s.find_all("div", attrs = {"class":"table-wrap"})

In [ ]:
items_listing = items[0].find_all("td", attrs = {"class":"list_time"})
PT_earthquakes_ref = []
for td in items_listing:
    # Extract href from the onclick attribute
    onclick_value = td['onclick']
    href = onclick_value.split("'")[1]
    PT_earthquakes_ref.append(href)



In [ ]:
burl+PT_earthquakes_ref[0]

In [ ]:
import time

PT_earthquake = dict()

for piece in range(len(PT_earthquakes_ref)):
    driver2 = webdriver.Chrome()
    driver2.get(burl+PT_earthquakes_ref[piece])
    html = driver2.page_source
    soup_s2 = BeautifulSoup(html)
    items = soup_s2.find('section', id='qDataSection')
    items2 = items.find_all('td')
    lines = []
    for line in items2:
        lines.append(line.get_text())
    lines = [lines[i] for i in range(1,len(lines),2)]
    date = lines[1]
    mag = lines[3]
    depth = lines[4]
    epicenter = " ".join(lines[5].split(" ")[0:3])
    mercali = lines[7].split('\xa0')
    entry = {
    date: {
        "magnitude": mag,
        "depth": depth,
        "epicenter": epicenter,
        "mercali": mercali
        }
    }
    PT_earthquake.update(entry)   
    time.sleep(0.5)    

In [ ]:
pt_ek = pd.DataFrame.from_dict(PT_earthquake)

In [ ]:
# Specify the save folder
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
directory_path = os.path.join(root_dir, 'dbs') # Change this to your folder path
os.makedirs(directory_path, exist_ok=True)
file_path = os.path.join(directory_path, "PT_earthquakes_strong_T.csv")
pt_ek.T.to_csv(file_path)